In [ ]:
from datetime import date
from glob import glob
import os, sys
from os.path import join, dirname
from os.path import getsize
import pickle
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import nilearn.image, nilearn.masking
from nilearn import plotting as nplt
import nilearn.decoding

import statsmodels.stats.multitest

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import PCA
# from PCRegression import PCR
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import random

In [ ]:
%matplotlib inline

In [ ]:
id_list = ['GA', 'GB']
nn_list = ['01', '02', '05', '07', '08', '11', '12', '13', '14', '15',
           '18', '19', '20', '21', '23', '26', '27', '28', '29', '30',
           '31', '32', '33', '34', '35', '36', '37', '38', '42', '44']
run_list = ['r01', 'r02', 'r03', 'r04', 'r05', 'r06']

In [ ]:
root_dir = '/Volumes/T7SSD1/GA'

In [ ]:
fMRI_dir = root_dir + '/fMRI_data'
preproc_dir = fMRI_dir + '/preproc_data'
stat_dir = fMRI_dir + '/stats'
roi_dir = fMRI_dir + '/roi'
loc_dir = roi_dir + '/localizer'
dmn_dir = roi_dir + '/DMN'

data_dir = preproc_dir

In [ ]:
def fast_masking(img, roi):
    # img : nifti image
    # roi : nifti image
    # output : (times, voxels)-dimension fdata array
    img_data = img.get_fdata()
    roi_mask = roi.get_fdata().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T    # the shape is (times, voxels) which is to cross-validate for times(=runs)

In [ ]:
## LDA analysis
model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

In [ ]:
def cross_valid(estimator, ROI_imgs, pickle_name):
    # estimator : model
    # ROI_imgs : nifti image
    # pickle_name : string
    # output : A leave-one-run-out cross-validation (LORO-CV) result and save pickle file
    ## set the parameters
    nrun = 3
    ntpr = 96 # a number of trials per run
    cv = GroupKFold(nrun)
    y = [j for i in range(nrun) for j in target_pos] # answer
    group = [i for i in range(nrun) for j in target_pos] # run number
    
    ## cross-validation
    pkl = {}
    for ii in id_list:
        for nn in nn_list:
            subj = ii + nn
            for roi, roi_img in ROI_imgs.items():
                for i, pp in enumerate(['practiced','unpracticed']):
                    X = np.array([fast_masking(img=data[subj,rr],roi=roi_img) for rr in run_list[i*3:(i+1)*3]])
                    X = X.reshape([nrun*ntpr,X.shape[-1]],order='C') # X.shape = (ntrial,nvoxle)
                    score = cross_validate(
                        estimator=estimator
                        , X=X, y=y, groups=group
                        , cv=cv, return_estimator=True, return_train_score=True)
                    pkl[subj, roi, pp] = np.mean(score['test_score'])
                print(subj, roi, end='\r')
                
    ## Save as .pickle
    with open(today+'_%s.pkl'%pickle_name,"wb") as fw:
        pickle.dump(pkl, fw)
        
    return pkl

In [ ]:
def make_dataframe(roi_list, pkl):
    wit_df = pd.DataFrame(columns=['subj', 'roi_name', 'Mapping', 'Mean Accuracy', 'Stage'])

    for ii in id_list:
        ss = 'Early' if ii == 'GA' else 'Late'
        for nn in nn_list:
            subj = ii + nn
            for roi_name in roi_list:
                for pp in ['unpracticed', 'practiced']:
                    wit_df = wit_df.append(
                        {'subj': subj
                         ,'roi_name': roi_name
                         ,'Mapping': pp
                         ,'Mean Accuracy': np.mean(pkl[subj, roi_name, pp])
                         ,'Stage': ss}
                        , ignore_index=True)
    return wit_df

In [ ]:
def wit_df_t_test(wit_df, roi_list, column_name, criteria):
    gg = wit_df[column_name].unique()
    key = list(criteria.keys())[0]
    value = list(criteria.values())[0]
    
    ttest = {}
    res = {}

    for roi in roi_list:
        sub_df = wit_df[(wit_df['roi_name'] == roi) & (wit_df[key] == value)]

        mean_accs = [sub_df[sub_df[column_name] == i]['Mean Accuracy'] for i in gg]

        ttest[roi] = scipy.stats.ttest_rel(mean_accs[0], mean_accs[1])
        res[roi] = statsmodels.stats.multitest.fdrcorrection(ttest[roi].pvalue)

    return res

# MVPA
#### Multi-voxel pattern analysis (MVPA) is gaining increasing interest in the neuroimaging community because it allows to detect differences between conditions with higher sensitivity than conventional univariate analysis by focusing on the analysis and comparison of distributed patterns of activity. In such a multivariate approach, data from individual voxels within a region are jointly analyzed. Furthermore, MVPA is often presented in the context of "brain reading" applications reporting that specific mental states or representational content can be decoded from fMRI activity patterns after performing a "training" or "learning phase. In this context, MVPA tools are often referred to as classifiers or, more generally, learning machines. The latter names stress that many MVPA tools originate from a field called machine learning, a branch of artificial intelligence.

In [ ]:
## background image
img_bg = join(roi_dir,'mni152_2009bet.nii.gz')

# GLM: overall fMRI activites associated with the 4 targets

## Loading $\beta$s

In [ ]:
## load betas
data = {}

for ii in id_list:
    for nn in nn_list:
        subj = ii + nn
        print(subj, end='\r')
        for run in run_list:
            data[subj, run] = nilearn.image.load_img(join(data_dir,nn,'betasLSS.%s.%s.nii.gz'%(subj,run)))

## indexing
print('indexing...', end='\r')
for key, value in data.items():
    data[key] = nilearn.image.index_img(value, np.arange(1, 97))

## labeling with target position
target_pos = []

with open(join(root_dir,'targetID.txt')) as file:
    for line in file:
        target_pos.append(int(line.strip()))
        
target_pos = target_pos[1:97]
target_path = list(range(1,13))*8